<a href="https://colab.research.google.com/github/AlexandreXavierMartins/Sim/blob/main/simpy_aula_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpy

Em um banco, clientes chegam a cada 2 minutos seguindo uma distribuição exponencial. Dos clientes que chegam $20\%$ são clientes com atendimento preferencial, o restante vai para o caixa normal. O tempo médio de atendimento em qualquer caixa é de 6 minutos com desvio padrão de 30 segundos, seguindo uma distribuição normal. Simule por 50 dias, sem aquecimento, 24 horas por dia. Defina o número mínimo de atendentes em cada tipo de caixa de maneira que o tempo máximo de fila não ultrapasse 40 minutos. Conte quantos clientes são atendidos em cada caixa.

In [ ]:
import simpy
import numpy as np

# Definição dos parâmetros
tempo_chegada = 2  # Chegada de Cliente (minutos)
tempo_total = 50*24*60  # Tempo total da simulação (10 dias)
fila_normal = []
fila_preferencial = []
cont_normal = 0
cont_preferencial = 0
tempo_max = 0
cap_normal = 20
cap_preferencial = 1
imprime = 0

def atende_preferencial(cliente_id):
  global fila_preferencial, tempo_max, cont_preferencial
  if imprime:
    print(f"Chegada do Cliente Preferencial {cliente_id} em {env.now:.2f}s")
  if (cap_preferencial - atendente_preferencial.count) == 0:
    fila_preferencial.append(cliente_id)
    if imprime:
      print('Fila Pref.', fila_preferencial)
    tempo = env.now
  with atendente_preferencial.request() as req:
    yield req
    if len(fila_preferencial) > 0:
      fila_preferencial.pop(0)
      tempo = env.now - tempo
      if tempo > tempo_max:
        tempo_max = tempo
    if imprime:
      print('Fila Pref.', fila_preferencial)
      print(f"Cliente Preferencial {cliente_id} começa atendimento {env.now:.2f}s")
    yield env.timeout(np.random.normal(6, 0.5))
    if imprime:
      print(f"Cliente Preferencial {cliente_id} atendido em {env.now:.2f}s")
    cont_preferencial += 1


def atende_normal(cliente_id):
  global fila_normal, tempo_max, cont_normal
  if imprime:
    print(f"Chegada do Cliente Normal {cliente_id} em {env.now:.2f}s")

  if (cap_normal - atendente_normal.count) == 0:
    fila_normal.append(cliente_id)
    tempo = env.now
    if imprime:
      print('Fila Norm.', fila_normal)
  with atendente_normal.request() as req:
    yield req
    if len(fila_normal) > 0:
      fila_normal.pop(0)
      tempo = env.now - tempo
      if tempo > tempo_max:
        tempo_max = tempo
    if imprime:
      print('Fila Norm.', fila_normal)
      print(f"Cliente {cliente_id} começa atendimento {env.now:.2f}s")
    yield env.timeout(np.random.normal(6, 0.5))
    if imprime:
      print(f"Cliente {cliente_id} atendido em {env.now:.2f}s")
    cont_normal += 1

# Processo de chegada dos carros
def chegada_cliente(env):
    cliente_id = 0
    while True:
        cliente_id += 1
        #Sorteia qual caixa, 20% Preferencial e 80% normal
        x = np.random.uniform(0, 1)
        if x <= 0.2:
          env.process(atende_preferencial(cliente_id))
        else:
          env.process(atende_normal(cliente_id))
        #Gera nova chegada
        yield env.timeout(np.random.exponential(tempo_chegada))
# Inicialização da simulação
env = simpy.Environment()
atendente_normal = simpy.Resource(env, capacity=cap_normal)
atendente_preferencial = simpy.Resource(env, capacity=cap_preferencial)
env.process(chegada_cliente(env))
#env.run(until=200)
env.run(until=tempo_total)
print('Tempo máximo de fila:', tempo_max,'minutos')
print('Clientes atendidos em cada caixa:', cont_normal, cont_preferencial)

Tempo máximo de fila: 44.754215811844915 minutos
Clientes atendidos em cada caixa: 28879 7137


Atributo:

Produtos são transportados entre 2 postos de trabalho. O tempo de transporte é em função de seu peso. Sabe-se que os produtos pesam em média 100 kg com um desvio padrão de 5 kg, seguindo uma Normal. O tempo de transporte do produto é dado pela seguinte função $t(p) = p \times 0.33 + 5$ (minutos), onde p é o peso do produto, se o produto pesa mais que 100 quilos ele é enviado ao setor 1, senão é enviado ao setor 2. Somente um produto pode ser transportado por vez. O intervalo entre chegadas é de 50 minutos seguindo uma Exponencial. Construa um modelo de simulação e simule o funcionamento desse sistema por 400 horas. Verifique quantos produtos foram levados para cada setor.

In [7]:
import simpy
import numpy as np

tempo_total = 2*60
tempo_chegada = 50
imprime = 1

def transporte(produto_id, peso):
  global imprime
  if imprime:
    print(f"Chegada do produto {produto_id} com peso {peso} em {env.now:.2f}s")

  with transportador.request() as req:
    yield req
    yield env.timeout(peso*0.33 + 5)
    if imprime:
      print(f"Produto {produto_id} tranportado {env.now:.2f}s")




# Processo de chegada dos produtos
def chegada_produto(env):
    produto_id = 0
    while True:
        produto_id += 1
        #Sorteia o peso do produto entre Normal(100, 5)
        peso = np.random.normal(100, 5)
        if imprime:
          print(f"Peso do produto {produto_id}: {peso}")
        env.process(transporte(produto_id, peso))
        #Gera nova chegada
        yield env.timeout(np.random.exponential(tempo_chegada))


# Inicialização da simulação
env = simpy.Environment()
transportador = simpy.Resource(env, capacity=1)
env.process(chegada_produto(env))

env.run(until = tempo_total)

Peso do produto 1: 103.40601031616292
Chegada do produto 1 com peso 103.40601031616292 em 0.00s
Produto 1 tranportado 39.12s
Peso do produto 2: 95.04717523599385
Chegada do produto 2 com peso 95.04717523599385 em 40.85s
Peso do produto 3: 96.77136157098872
Chegada do produto 3 com peso 96.77136157098872 em 57.75s
Produto 2 tranportado 77.21s
Produto 3 tranportado 114.15s


Variável:

Carros chegam a um posto de combustível a cada 15 minutos seguindo uma distribuição exponencial. Cada carro tem uma demanda média de 25 litros com desvio padrão de 3 litros. Caso não tenha combustível suficiente o carro deixa o posto. O combustível chega a cada 10 dias em média com um desvio padrão de 2 dias e completa o estoque do posto, que tem capacidade de 1000 litros. A demanda de cada carro segue uma distribuição normal. Desconsidere o tempo de abastecimento. Simule por 100 dias, 24 horas por dia. Conte o número de carros abastecidos e não abastecidos. Coloque na tela o valor atual do estoque e o número de carros abastecidos e não abastecidos. Colete os dados em dias. Verifique se o sistema está bem dimensionado.

In [ ]:
import simpy
import numpy as np

tempo_total = 24*60*100
tempo_chegada = 15
imprime = 0
cont = 0
Estoque = 10000

def chegada_estoque(env):
  global Estoque
  while True:
    yield env.timeout(np.random.normal(10*24*60, 2*24*60))
    Estoque = 10000

def abastece(carro_id, demanda):
  global imprime, Estoque, cont
  yield env.timeout(0)
  if demanda <= Estoque:
    Estoque -= demanda
    #print(f"Carro {carro_id} abastecido {env.now:.2f}s")
    print('Estoque atual:', Estoque)
  else:
    cont += 1
    if imprime:
      print(f"Carro {carro_id} não abastecido {env.now:.2f}s")




# Processo de chegada dos produtos
def chegada_carros(env):
    carro_id = 0
    global Estoque, imprime
    while True:
        carro_id += 1
        #Sorteia o peso do produto entre Normal(100, 5)
        demanda = np.random.normal(25, 3)
        if imprime:
          print(f"Demanda do carro {carro_id}: {demanda}")
        env.process(abastece(carro_id, demanda))
        #Gera nova chegada
        yield env.timeout(np.random.exponential(tempo_chegada))


# Inicialização da simulação
env = simpy.Environment()
#frentista = simpy.Resource(env, capacity=1)
env.process(chegada_carros(env))
env.process(chegada_estoque(env))

env.run(until = tempo_total)
print("Carros não abastecidos:", cont)

Considere um consultório de pronto atendimento de um hospital em que 60$\%$ do pacientes são de prioridade baixa (pulseira verde), 25$\%$ de prioridade intermediária (pulseira amarela) e o restante de prioridade alta (pulseira vermelha). Existem 2 médicos que realizam os atendimentos e que sempre verificam inicialmente a ordem de prioridade dos pacientes na fila. Os pacientes chegam entre si em intervalos exponencialmente distribuídos, com média de 5 minutos e o atendimento é também exponencialmente distribuído, com média de 9 minutos por paciente.

In [ ]:
import simpy
import numpy as np

tempo_total = 3*60
tempo_chegada = 5
imprime = 1
fila = []
cap = 2


def atendimento(paciente_id, prioridade):
  global imprime, cap
  if medicos.count == cap:
      fila.append([paciente_id, prioridade])
      if imprime:
        print('Fila:', fila)
  with medicos.request(priority=prioridade) as req:
    yield req
    if imprime:
      print(f'Médico começa atendimento do paciente {paciente_id}, prioridade {prioridade} em {env.now}')
      if len(fila) > 0:
        fila.remove([paciente_id, prioridade])
        if imprime:
          print('Fila:', fila)
    yield env.timeout(np.random.exponential(9))
    if imprime:
      print(f'Termina o atendimento do paciente {paciente_id} - {env.now}')


# Processo de chegada dos produtos
def chegada_paciente(env):
    paciente_id = 0
    global imprime
    while True:
        paciente_id += 1
        #Sorteia o nível de prioridade entre 0 e 1
        r = np.random.uniform(0, 1)
        if r <= 0.6:
          prioridade = 2
        elif r <= 0.85:
          prioridade = 1
        else:
          prioridade = 0
        if imprime:
          print(f"Paciente: {paciente_id}, Prioridade: {prioridade} - {env.now}")
        env.process(atendimento(paciente_id, prioridade))
        #Gera nova chegada
        yield env.timeout(np.random.exponential(tempo_chegada))


# Inicialização da simulação
env = simpy.Environment()
medicos = simpy.PriorityResource(env, capacity=cap)
env.process(chegada_paciente(env))

env.run(until = tempo_total)
